In [4]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Function to load and interpret the CSV file
def load_and_interpret_gps_data(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)
    
    df.columns = ['timestamp', 'latitude', 'longitude', 'fix', 'num_satellites', 'hdop', 'altitude', 'geoidal_separation']
    
    # Convert the timestamp to a datetime object
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    
    # Filter out rows where latitude and longitude are both 0
    df = df[(df['latitude'] != 0) & (df['longitude'] != 0)]
    
    return df

# Function to plot the GPS data on an interactive map with a line connecting the points
def plot_gps_data_on_interactive_map(df):
    # Create a map centered around the mean latitude and longitude
    m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10)
    
    # Add a marker cluster to manage the markers
    marker_cluster = MarkerCluster().add_to(m)
    
    # Loop through each row in the DataFrame and add a marker to the map with a popup
    for index, row in df.iterrows():
        folium.Marker(
            [row['latitude'], row['longitude']],
            popup=f"Timestamp: {row['timestamp']}"
        ).add_to(marker_cluster)
    
    # Create a list of coordinates for the line
    coordinates = [[row['latitude'], row['longitude']] for index, row in df.iterrows()]
    
    # Add a line connecting the points
    folium.PolyLine(coordinates, color='blue', weight=2.5, opacity=1).add_to(m)
    
    # Save the map to an HTML file
    m.save('C:/Disco/Trabalho/Universidade/Mestrado/2ºano/2ºSemestre/Tese/Testes/GPS/interactive_map_with_line_and_popup.html')

# Main script
if __name__ == "__main__":
    # Specify the path to your CSV file
    file_path = 'C:/Disco/Trabalho/Universidade/Mestrado/2ºano/2ºSemestre/Tese/Testes/GPS/costanova-aveiro.csv'
    
    # Load and interpret the GPS data
    df = load_and_interpret_gps_data(file_path)
    
    # Plot the GPS data on an interactive map with a line connecting the points and popups for timestamps
    plot_gps_data_on_interactive_map(df)


In [14]:
df

,timestamp,latitude,longitude,fix,num_satellites,hdop,altitude,geoidal_separation
2,2024-03-12 18:04:49,40.609845,-8.749958,1,5,1.80,22.8,51.1
3,2024-03-12 18:04:56,40.609885,-8.749698,1,4,3.21,-0.2,51.1
4,2024-03-12 18:05:07,40.610382,-8.750183,1,5,1.80,46.5,51.1
6,2024-03-12 18:05:21,40.610607,-8.749928,1,4,3.18,32.6,51.1
8,2024-03-12 18:05:34,40.609882,-8.749185,1,6,1.73,3.6,51.1
...,...,...,...,...,...,...,...,...
599,2024-03-12 19:11:17,40.610090,-8.749747,1,4,2.09,41.5,51.1
600,2024-03-12 19:11:24,40.610137,-8.749895,1,4,2.09,32.3,51.1
601,2024-03-12 19:11:34,40.610423,-8.750948,1,4,2.09,0.1,51.1
603,2024-03-12 19:11:46,40.610727,-8.750867,1,5,2.00,-0.1,51.1


In [21]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Function to load and interpret the CSV file
def load_and_interpret_gps_data(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)
    
    # Ensure the columns are correctly named
    df.columns = ['timestamp', 'latitude', 'longitude', 'fix', 'num_satellites', 'hdop', 'altitude', 'geoidal_separation']
    
    # Convert the timestamp to a datetime object
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    
    # Filter out rows where latitude and longitude are both 0
    df = df[(df['latitude'] != 0) & (df['longitude'] != 0)]
    
    return df

# Function to apply moving average smoothing to the GPS data
def apply_moving_average_smoothing(df, window_size=5):
    # Ensure the DataFrame is sorted by timestamp
    df = df.sort_values(by='timestamp')
    
    # Apply moving average smoothing to latitude and longitude
    df['latitude_smoothed'] = df['latitude'].rolling(window=window_size).mean()
    df['longitude_smoothed'] = df['longitude'].rolling(window=window_size).mean()
    
    # Fill NaN values in the smoothed columns with the original values
    df['latitude_smoothed'].fillna(df['latitude'], inplace=True)
    df['longitude_smoothed'].fillna(df['longitude'], inplace=True)
    
    return df


# Function to plot the GPS data on an interactive map with a line connecting the points
def plot_gps_data_on_interactive_map(df):
    # Apply moving average smoothing to the GPS data
    df_smoothed = apply_moving_average_smoothing(df)
    
    # Create a map centered around the mean latitude and longitude of the smoothed data
    m = folium.Map(location=[df_smoothed['latitude_smoothed'].mean(), df_smoothed['longitude_smoothed'].mean()], zoom_start=10)
    
    # Add a marker cluster to manage the markers
    marker_cluster = MarkerCluster().add_to(m)
    
    # Loop through each row in the smoothed DataFrame and add a marker to the map with a popup
    for index, row in df_smoothed.iterrows():
        folium.Marker(
            [row['latitude_smoothed'], row['longitude_smoothed']],
            popup=f"Timestamp: {row['timestamp']}"
        ).add_to(marker_cluster)
    
    # Create a list of coordinates for the line
    coordinates = [[row['latitude_smoothed'], row['longitude_smoothed']] for index, row in df_smoothed.iterrows()]
    
    # Add a line connecting the smoothed points
    folium.PolyLine(coordinates, color='blue', weight=2.5, opacity=1).add_to(m)
    
    # Save the map to an HTML file
    m.save('C:/Disco/Trabalho/Universidade/Mestrado/2ºano/2ºSemestre/Tese/Testes/GPS/interactive_map_with_smoothed_line_and_popup.html')

# Main script
if __name__ == "__main__":
    # Specify the path to your CSV file
    file_path = 'C:/Disco/Trabalho/Universidade/Mestrado/2ºano/2ºSemestre/Tese/Testes/GPS/costanova-vagueira.csv'
    
    # Load and interpret the GPS data
    df = load_and_interpret_gps_data(file_path)
    print(df)
    
    # Plot the GPS data on an interactive map with a line connecting the smoothed points and popups for timestamps
    plot_gps_data_on_interactive_map(df)


              timestamp   latitude  longitude  fix  num_satellites  hdop  \
2   2024-03-12 18:04:49  40.609845  -8.749958    1               5  1.80   
3   2024-03-12 18:04:56  40.609885  -8.749698    1               4  3.21   
4   2024-03-12 18:05:07  40.610382  -8.750183    1               5  1.80   
6   2024-03-12 18:05:21  40.610607  -8.749928    1               4  3.18   
8   2024-03-12 18:05:34  40.609882  -8.749185    1               6  1.73   
..                  ...        ...        ...  ...             ...   ...   
599 2024-03-12 19:11:17  40.610090  -8.749747    1               4  2.09   
600 2024-03-12 19:11:24  40.610137  -8.749895    1               4  2.09   
601 2024-03-12 19:11:34  40.610423  -8.750948    1               4  2.09   
603 2024-03-12 19:11:46  40.610727  -8.750867    1               5  2.00   
604 2024-03-12 19:11:52  40.610462  -8.750627    1               4  2.24   

     altitude  geoidal_separation  
2        22.8                51.1  
3        -0.2  